###  Let's write an example of concurrent write and read to a Azure queue as an example

### 1. We create a queue 

In [23]:
from azure.storage.queue import (
        QueueClient,
        BinaryBase64EncodePolicy,
        BinaryBase64DecodePolicy
)

import os, uuid

# Retrieve the connection string from an environment
# variable named AZURE_STORAGE_CONNECTION_STRING
connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")

# Create a unique name for the queue
queue_name = "queue-" + "tdu"

# Instantiate a QueueClient object which will
# be used to create and manipulate the queue
print("Creating queue: " + queue_name)
queue_client = QueueClient.from_connection_string(connect_str, queue_name)

# Create the queue
# queue_client.create_queue()


Creating queue: queue-tdu


## 2. Create a thread pool randomly publish message to the queue 
This can be ran in a different process with multiple thread to publish

In [35]:
from concurrent.futures import ThreadPoolExecutor
import threading, itertools

import random
def uniqueid():
    while True:
        seed = str(uuid.uuid4())
        yield seed
        
def task(uuid):
    message = " " + uuid
    queue_client.send_message(message)

unique_sequence = uniqueid()
ids = list(itertools.islice(unique_sequence, 3000))

with ThreadPoolExecutor(max_workers=50) as pool:
    pool.map(task, ids)


# 3. Create a consumer to consume and delete all the messages we put 

You can hold this step and inspect what the queue look like in Azure Storage Explore under your subscription -> queues -> queue.name

In [38]:
# The example code in azure storage official tutorial will delete all record.
# One thing I realize is the queue doesn't really be in order.

'''
    bisibility_timeout is the "lock" of a record to make sure the "uniqueness"
'''
messages = queue_client.receive_messages(messages_per_page=5, visibility_timeout=5*60)

for msg_batch in messages.by_page():
    for msg in msg_batch:
        print("Batch dequeue message: " + msg.content)
        queue_client.delete_message(msg)